In [12]:
import bs4
import requests
import json
# deptID = {
#     '人文社會科學院':'****11;****12;****15;****17;****18',
#     '人文社會科學院共同課程':'****11;****12;****15;****17;****18', # <------- 限修的學號
#     '西洋語文學系':'****11',
#     '運動健康與休閒學系':'****12',
#     '東亞語文學系':'****17',
#     '運動競技學系':'****18',
#     '建築學系':'****15', # warning i dont know new id
#     '工藝與創意設計學系':'****15', # warning i dont know new id
#     '創意設計與建築學系':'****15', # warning i dont know new id
#     # hi
#     '法學院':'****21;****22;****23',
#     '法學院共同課程':'****21;****22;****23',
#     '法律學系':'****21',
#     '政治法律學系':'****22',
#     '財經法律學系':'****23',
#     # hi
#     '管理學院':'****31;****71;****32;****33;****33;****33',
#     '管理學院共同課程':'****31;****71;****32;****33;****33;****33',
#     '應用經濟學系':'****31',
#     '亞太工商管理學系':'****71',
#     '金融管理學系':'****32',
#     '資訊管理學系':'****33',
#     # hi
#     '理學院':'****41;****61;****42;****43',
#     '理學院共同課程':'****41;****61;****42;****43',
#     '應用數學系':'****41',
#     '生命科學系':'****61',
#     '應用化學系':'****42',
#     '應用物理學系':'****43',
#     # hi
#     '工學院':'****51;****52;****56;****55',
#     '工學院共同課程':'****51;****52;****56;****55',
#     '電機工程學系':'****51',
#     '土木與環境工程學系':'****52',
#     '化學工程及材料工程學系':'****56',
#     '資訊工程學系':'****55'
#     }

currentYear = 109
currentSemester = 1
coreGeneralEduV2 = {} # 核心通識分類
data = '<tr><td+width=""33%"">開課學年：' + str(currentYear) + '　　開課學期：第' + str(currentSemester) + '學期</td><td+width=""33%"">開課部別：大學部</td><td+width=""34%"">開課系所：無</td></tr><tr><td+width=""33%"">開課班級：無</td><td+width=""33%"">授課教師：無</td><td+width=""34%"">上課時間：無</td></tr>'
req = requests.post('https://course.nuk.edu.tw/QueryCourse/QueryResult.asp', data = {'Condition':data,'Flag': 1,'OpenYear': currentYear,'Helf': currentSemester,'Pclass': 'A','Sclass': '','Yclass': '','SirName': '','Sirno': '','WeekDay': '','Card': '','Subject': '','Language': '','Pre_Cono': '','Coname': ''})
req.encoding = 'big5'
# print(req.text)
soup = bs4.BeautifulSoup(req.text,'html.parser')
soup = soup.find_all('tr')
# print(soup.prettify())
i = 0
all_course = [] #總課表

# coreGeneralEdu = [] # 核心通識分類

for ele in soup:
    if i<=3:
        i+=1
        continue
    tdSet = ele.find_all('td')

    #TODO 微學分奇怪的格式
    if float(tdSet[6].text.upper()) < 1.0 and float(tdSet[6].text.upper()) > 0.0:
        continue
    tempCourse = {}
    tempCourse['id'] = tdSet[0].text.upper() + tdSet[1].text.upper()
    tempCourse['dept'] = tdSet[0].text.upper()
    tempCourse['name'] = tdSet[5].text
    tempCourse['grade'] = tdSet[3].text
    tempCourse['class'] = tdSet[4].text
    tempCourse['teacher'] = tdSet[12].decode_contents()[0:-5].replace('<br/>',',')
    # print(tdSet[12].decode_contents()[0:-5].replace('<br/>',','))
    
    course_time = []
    tempCourseTime = []
    for i in range(14,21):
        if tdSet[i].text != '\u3000':
            # print(ele.text.split(' ')[i])
            for day in tdSet[i].text.split(','):
                # print(i-13, day)
                if day == 'X':
                    course_time.append([i-13, 0])
                elif day == 'Y':
                    course_time.append([i-13, 4.5])
                else:
                    try:
                        course_time.append([i-13, int(day)])
                    except ValueError:
                        print("Convert to int error: {}".format(day))
                        pass
    
    tempCourse['period'] = course_time
    if tdSet[7].text == '必':
        tempCourse['category'] = True
    else:
        tempCourse['category'] = False
    tempCourse['credit'] = tdSet[6].text
    tempCourse['location'] = tdSet[13].text
    # tempCourse['limit'] = tdSet[21].text
    #tempCourse['limit'] = []
    
    
#     if '限修' in tdSet[21].text:
#         limitList = ''
#         limitReq = requests.get('https://course.nuk.edu.tw/QueryCourse/Limit.asp?OpenYear='+str(currentYear)+'&Helf='+str(currentSemester)+'&Sclass='+tempCourse['dept']+'&Cono='+tempCourse['id'])
#         limitReq.encoding = 'big5'
#         limitSoup = bs4.BeautifulSoup(limitReq.text, 'html.parser')
#         limitSoup = limitSoup.select('table[border="0"] td[width="80%"]')
#         # print(tempCourse['name'])
#         for ele in limitSoup:
#             # print(ele.text)
#             if '年級' in ele.text:
#                 limitList+=ele.text[-7:-1]
#             else:
#                 try:
#                     limitList+=deptID[ele.text]
#                 except:
#                     limitList+=ele.text.split(' ')[0]
#             limitList+=';'
#         tempCourse['limit'] = limitList.split(';')[:-1]

    # 放入字典
    all_course.append(tempCourse)
    
    # print(all_course)
# for r in coreGeneralEdu:
#     print(r)

# print(all_course)
# 存檔
with open('AllCourse.json','w',encoding='utf8') as f:
    f.write(json.dumps(all_course, ensure_ascii=False).encode("utf8",errors='ignore').decode("utf8",errors='ignore'))

In [6]:
coreGeneralEduV2 = {} # 核心通識分類
for y in range(103,109):
    for semester in range(1,3):
        print('現在進度', y, semester)
        data = '<tr><td+width=""33%"">開課學年：' + str(y) + '　　開課學期：第' + str(semester) + '學期</td><td+width=""33%"">開課部別：大學部</td><td+width=""34%"">開課系所：無</td></tr><tr><td+width=""33%"">開課班級：無</td><td+width=""33%"">授課教師：無</td><td+width=""34%"">上課時間：無</td></tr>'
        req = requests.post('https://course.nuk.edu.tw/QueryCourse/QueryResult.asp', data = {'Condition':data,'Flag': 1,'OpenYear': y,'Helf': semester,'Pclass': 'A','Sclass': '','Yclass': '','SirName': '','Sirno': '','WeekDay': '','Card': '','Subject': '','Language': '','Pre_Cono': '','Coname': ''})
        req.encoding = 'big5'
        soup = bs4.BeautifulSoup(req.text,'html.parser')
        soup = soup.find_all('tr')
        i = 0
        all_course = [] #總課表
        for ele in soup:
            if i<=3:
                i+=1
                continue
            tdSet = ele.find_all('td')
            tempCourse = {}
            tempCourse['dept'] = tdSet[0].text.upper()
            tempCourse['id'] = tdSet[1].text.upper()
            tempCourse['target'] = tdSet[3].text
            tempCourse['name'] = tdSet[5].text
            all_course.append(tempCourse)
        for ele in all_course:
            if ele['dept'] == 'CC':
                # print(ele['id'])
                infoUrl = 'https://course.nuk.edu.tw/QueryCourse/tcontent.asp?OpenYear=' + str(y) + '&Helf=' + str(semester) + '&Sclass=CC&Cono=' + ele['id']
                CCinfo = requests.get(infoUrl)
                CCinfo.encoding = 'big5'
                CCinfoSoup = bs4.BeautifulSoup(CCinfo.text,'html.parser')
                # td[11]是通識分類 取出之後為"科學素養－科學素養" 再將其切割為"科學素養"
                CCinfoSoup = CCinfoSoup.find_all('td')[11].text.split('－')[0]
                coreGeneralEduV2['CC'+ele['id'] +'Y'+str(y) + str(semester)] = [ele['name'].strip(), CCinfoSoup]
        print(coreGeneralEduV2)
        with open('GeneralEduList'+str(y)+'_'+str(semester)+'.json','w',encoding='utf8') as f:
            f.write(json.dumps(coreGeneralEduV2, ensure_ascii=False).encode("utf8",errors='ignore').decode("utf8",errors='ignore'))

現在進度 103 1
{'CCC511Y1031': ['科普名著導讀', '科學素養'], 'CCC512Y1031': ['科普讀物導讀', '科學素養'], 'CCC514Y1031': ['科普讀物導讀', '科學素養'], 'CCC541Y1031': ['科技與社會', '科學素養'], 'CCC542Y1031': ['科技與社會', '科學素養'], 'CCC543Y1031': ['科技與社會', '科學素養'], 'CCC544Y1031': ['科技與社會', '科學素養'], 'CCC611Y1031': ['環境倫理', '倫理素養'], 'CCC631Y1031': ['科技倫理', '倫理素養'], 'CCC632Y1031': ['科技倫理', '倫理素養'], 'CCC651Y1031': ['企業倫理', '倫理素養'], 'CCC652Y1031': ['企業倫理', '倫理素養'], 'CCC671Y1031': ['職場倫理', '倫理素養'], 'CCC672Y1031': ['職場倫理', '倫理素養'], 'CCI111Y1031': ['邏輯思維與論證', '思維方法'], 'CCI112Y1031': ['邏輯思維與論證', '思維方法'], 'CCI113Y1031': ['邏輯思維與論證', '思維方法'], 'CCI121Y1031': ['哲學基本問題', '思維方法'], 'CCI122Y1031': ['哲學基本問題', '思維方法'], 'CCI211Y1031': ['藝術概論', '美學素養'], 'CCI212Y1031': ['藝術概論', '美學素養'], 'CCI213Y1031': ['藝術概論', '美學素養'], 'CCI251Y1031': ['台灣藝術史', '美學素養'], 'CCI261Y1031': ['西洋藝術史概論', '美學素養'], 'CCI262Y1031': ['西洋藝術史概論', '美學素養'], 'CCO311Y1031': ['法律與人生', '公民素養'], 'CCO312Y1031': ['法律與人生', '公民素養'], 'CCO351Y1031': ['服務學習', '公民素養'], 'CCO352Y1031': ['服務學習', '公民素養'],

{'CCC511Y1031': ['科普名著導讀', '科學素養'], 'CCC512Y1031': ['科普讀物導讀', '科學素養'], 'CCC514Y1031': ['科普讀物導讀', '科學素養'], 'CCC541Y1031': ['科技與社會', '科學素養'], 'CCC542Y1031': ['科技與社會', '科學素養'], 'CCC543Y1031': ['科技與社會', '科學素養'], 'CCC544Y1031': ['科技與社會', '科學素養'], 'CCC611Y1031': ['環境倫理', '倫理素養'], 'CCC631Y1031': ['科技倫理', '倫理素養'], 'CCC632Y1031': ['科技倫理', '倫理素養'], 'CCC651Y1031': ['企業倫理', '倫理素養'], 'CCC652Y1031': ['企業倫理', '倫理素養'], 'CCC671Y1031': ['職場倫理', '倫理素養'], 'CCC672Y1031': ['職場倫理', '倫理素養'], 'CCI111Y1031': ['邏輯思維與論證', '思維方法'], 'CCI112Y1031': ['邏輯思維與論證', '思維方法'], 'CCI113Y1031': ['邏輯思維與論證', '思維方法'], 'CCI121Y1031': ['哲學基本問題', '思維方法'], 'CCI122Y1031': ['哲學基本問題', '思維方法'], 'CCI211Y1031': ['藝術概論', '美學素養'], 'CCI212Y1031': ['藝術概論', '美學素養'], 'CCI213Y1031': ['藝術概論', '美學素養'], 'CCI251Y1031': ['台灣藝術史', '美學素養'], 'CCI261Y1031': ['西洋藝術史概論', '美學素養'], 'CCI262Y1031': ['西洋藝術史概論', '美學素養'], 'CCO311Y1031': ['法律與人生', '公民素養'], 'CCO312Y1031': ['法律與人生', '公民素養'], 'CCO351Y1031': ['服務學習', '公民素養'], 'CCO352Y1031': ['服務學習', '公民素養'], 'CCO354Y10

{'CCC511Y1031': ['科普名著導讀', '科學素養'], 'CCC512Y1031': ['科普讀物導讀', '科學素養'], 'CCC514Y1031': ['科普讀物導讀', '科學素養'], 'CCC541Y1031': ['科技與社會', '科學素養'], 'CCC542Y1031': ['科技與社會', '科學素養'], 'CCC543Y1031': ['科技與社會', '科學素養'], 'CCC544Y1031': ['科技與社會', '科學素養'], 'CCC611Y1031': ['環境倫理', '倫理素養'], 'CCC631Y1031': ['科技倫理', '倫理素養'], 'CCC632Y1031': ['科技倫理', '倫理素養'], 'CCC651Y1031': ['企業倫理', '倫理素養'], 'CCC652Y1031': ['企業倫理', '倫理素養'], 'CCC671Y1031': ['職場倫理', '倫理素養'], 'CCC672Y1031': ['職場倫理', '倫理素養'], 'CCI111Y1031': ['邏輯思維與論證', '思維方法'], 'CCI112Y1031': ['邏輯思維與論證', '思維方法'], 'CCI113Y1031': ['邏輯思維與論證', '思維方法'], 'CCI121Y1031': ['哲學基本問題', '思維方法'], 'CCI122Y1031': ['哲學基本問題', '思維方法'], 'CCI211Y1031': ['藝術概論', '美學素養'], 'CCI212Y1031': ['藝術概論', '美學素養'], 'CCI213Y1031': ['藝術概論', '美學素養'], 'CCI251Y1031': ['台灣藝術史', '美學素養'], 'CCI261Y1031': ['西洋藝術史概論', '美學素養'], 'CCI262Y1031': ['西洋藝術史概論', '美學素養'], 'CCO311Y1031': ['法律與人生', '公民素養'], 'CCO312Y1031': ['法律與人生', '公民素養'], 'CCO351Y1031': ['服務學習', '公民素養'], 'CCO352Y1031': ['服務學習', '公民素養'], 'CCO354Y10

{'CCC511Y1031': ['科普名著導讀', '科學素養'], 'CCC512Y1031': ['科普讀物導讀', '科學素養'], 'CCC514Y1031': ['科普讀物導讀', '科學素養'], 'CCC541Y1031': ['科技與社會', '科學素養'], 'CCC542Y1031': ['科技與社會', '科學素養'], 'CCC543Y1031': ['科技與社會', '科學素養'], 'CCC544Y1031': ['科技與社會', '科學素養'], 'CCC611Y1031': ['環境倫理', '倫理素養'], 'CCC631Y1031': ['科技倫理', '倫理素養'], 'CCC632Y1031': ['科技倫理', '倫理素養'], 'CCC651Y1031': ['企業倫理', '倫理素養'], 'CCC652Y1031': ['企業倫理', '倫理素養'], 'CCC671Y1031': ['職場倫理', '倫理素養'], 'CCC672Y1031': ['職場倫理', '倫理素養'], 'CCI111Y1031': ['邏輯思維與論證', '思維方法'], 'CCI112Y1031': ['邏輯思維與論證', '思維方法'], 'CCI113Y1031': ['邏輯思維與論證', '思維方法'], 'CCI121Y1031': ['哲學基本問題', '思維方法'], 'CCI122Y1031': ['哲學基本問題', '思維方法'], 'CCI211Y1031': ['藝術概論', '美學素養'], 'CCI212Y1031': ['藝術概論', '美學素養'], 'CCI213Y1031': ['藝術概論', '美學素養'], 'CCI251Y1031': ['台灣藝術史', '美學素養'], 'CCI261Y1031': ['西洋藝術史概論', '美學素養'], 'CCI262Y1031': ['西洋藝術史概論', '美學素養'], 'CCO311Y1031': ['法律與人生', '公民素養'], 'CCO312Y1031': ['法律與人生', '公民素養'], 'CCO351Y1031': ['服務學習', '公民素養'], 'CCO352Y1031': ['服務學習', '公民素養'], 'CCO354Y10

{'CCC511Y1031': ['科普名著導讀', '科學素養'], 'CCC512Y1031': ['科普讀物導讀', '科學素養'], 'CCC514Y1031': ['科普讀物導讀', '科學素養'], 'CCC541Y1031': ['科技與社會', '科學素養'], 'CCC542Y1031': ['科技與社會', '科學素養'], 'CCC543Y1031': ['科技與社會', '科學素養'], 'CCC544Y1031': ['科技與社會', '科學素養'], 'CCC611Y1031': ['環境倫理', '倫理素養'], 'CCC631Y1031': ['科技倫理', '倫理素養'], 'CCC632Y1031': ['科技倫理', '倫理素養'], 'CCC651Y1031': ['企業倫理', '倫理素養'], 'CCC652Y1031': ['企業倫理', '倫理素養'], 'CCC671Y1031': ['職場倫理', '倫理素養'], 'CCC672Y1031': ['職場倫理', '倫理素養'], 'CCI111Y1031': ['邏輯思維與論證', '思維方法'], 'CCI112Y1031': ['邏輯思維與論證', '思維方法'], 'CCI113Y1031': ['邏輯思維與論證', '思維方法'], 'CCI121Y1031': ['哲學基本問題', '思維方法'], 'CCI122Y1031': ['哲學基本問題', '思維方法'], 'CCI211Y1031': ['藝術概論', '美學素養'], 'CCI212Y1031': ['藝術概論', '美學素養'], 'CCI213Y1031': ['藝術概論', '美學素養'], 'CCI251Y1031': ['台灣藝術史', '美學素養'], 'CCI261Y1031': ['西洋藝術史概論', '美學素養'], 'CCI262Y1031': ['西洋藝術史概論', '美學素養'], 'CCO311Y1031': ['法律與人生', '公民素養'], 'CCO312Y1031': ['法律與人生', '公民素養'], 'CCO351Y1031': ['服務學習', '公民素養'], 'CCO352Y1031': ['服務學習', '公民素養'], 'CCO354Y10

{'CCC511Y1031': ['科普名著導讀', '科學素養'], 'CCC512Y1031': ['科普讀物導讀', '科學素養'], 'CCC514Y1031': ['科普讀物導讀', '科學素養'], 'CCC541Y1031': ['科技與社會', '科學素養'], 'CCC542Y1031': ['科技與社會', '科學素養'], 'CCC543Y1031': ['科技與社會', '科學素養'], 'CCC544Y1031': ['科技與社會', '科學素養'], 'CCC611Y1031': ['環境倫理', '倫理素養'], 'CCC631Y1031': ['科技倫理', '倫理素養'], 'CCC632Y1031': ['科技倫理', '倫理素養'], 'CCC651Y1031': ['企業倫理', '倫理素養'], 'CCC652Y1031': ['企業倫理', '倫理素養'], 'CCC671Y1031': ['職場倫理', '倫理素養'], 'CCC672Y1031': ['職場倫理', '倫理素養'], 'CCI111Y1031': ['邏輯思維與論證', '思維方法'], 'CCI112Y1031': ['邏輯思維與論證', '思維方法'], 'CCI113Y1031': ['邏輯思維與論證', '思維方法'], 'CCI121Y1031': ['哲學基本問題', '思維方法'], 'CCI122Y1031': ['哲學基本問題', '思維方法'], 'CCI211Y1031': ['藝術概論', '美學素養'], 'CCI212Y1031': ['藝術概論', '美學素養'], 'CCI213Y1031': ['藝術概論', '美學素養'], 'CCI251Y1031': ['台灣藝術史', '美學素養'], 'CCI261Y1031': ['西洋藝術史概論', '美學素養'], 'CCI262Y1031': ['西洋藝術史概論', '美學素養'], 'CCO311Y1031': ['法律與人生', '公民素養'], 'CCO312Y1031': ['法律與人生', '公民素養'], 'CCO351Y1031': ['服務學習', '公民素養'], 'CCO352Y1031': ['服務學習', '公民素養'], 'CCO354Y10

{'CCC511Y1031': ['科普名著導讀', '科學素養'], 'CCC512Y1031': ['科普讀物導讀', '科學素養'], 'CCC514Y1031': ['科普讀物導讀', '科學素養'], 'CCC541Y1031': ['科技與社會', '科學素養'], 'CCC542Y1031': ['科技與社會', '科學素養'], 'CCC543Y1031': ['科技與社會', '科學素養'], 'CCC544Y1031': ['科技與社會', '科學素養'], 'CCC611Y1031': ['環境倫理', '倫理素養'], 'CCC631Y1031': ['科技倫理', '倫理素養'], 'CCC632Y1031': ['科技倫理', '倫理素養'], 'CCC651Y1031': ['企業倫理', '倫理素養'], 'CCC652Y1031': ['企業倫理', '倫理素養'], 'CCC671Y1031': ['職場倫理', '倫理素養'], 'CCC672Y1031': ['職場倫理', '倫理素養'], 'CCI111Y1031': ['邏輯思維與論證', '思維方法'], 'CCI112Y1031': ['邏輯思維與論證', '思維方法'], 'CCI113Y1031': ['邏輯思維與論證', '思維方法'], 'CCI121Y1031': ['哲學基本問題', '思維方法'], 'CCI122Y1031': ['哲學基本問題', '思維方法'], 'CCI211Y1031': ['藝術概論', '美學素養'], 'CCI212Y1031': ['藝術概論', '美學素養'], 'CCI213Y1031': ['藝術概論', '美學素養'], 'CCI251Y1031': ['台灣藝術史', '美學素養'], 'CCI261Y1031': ['西洋藝術史概論', '美學素養'], 'CCI262Y1031': ['西洋藝術史概論', '美學素養'], 'CCO311Y1031': ['法律與人生', '公民素養'], 'CCO312Y1031': ['法律與人生', '公民素養'], 'CCO351Y1031': ['服務學習', '公民素養'], 'CCO352Y1031': ['服務學習', '公民素養'], 'CCO354Y10

{'CCC511Y1031': ['科普名著導讀', '科學素養'], 'CCC512Y1031': ['科普讀物導讀', '科學素養'], 'CCC514Y1031': ['科普讀物導讀', '科學素養'], 'CCC541Y1031': ['科技與社會', '科學素養'], 'CCC542Y1031': ['科技與社會', '科學素養'], 'CCC543Y1031': ['科技與社會', '科學素養'], 'CCC544Y1031': ['科技與社會', '科學素養'], 'CCC611Y1031': ['環境倫理', '倫理素養'], 'CCC631Y1031': ['科技倫理', '倫理素養'], 'CCC632Y1031': ['科技倫理', '倫理素養'], 'CCC651Y1031': ['企業倫理', '倫理素養'], 'CCC652Y1031': ['企業倫理', '倫理素養'], 'CCC671Y1031': ['職場倫理', '倫理素養'], 'CCC672Y1031': ['職場倫理', '倫理素養'], 'CCI111Y1031': ['邏輯思維與論證', '思維方法'], 'CCI112Y1031': ['邏輯思維與論證', '思維方法'], 'CCI113Y1031': ['邏輯思維與論證', '思維方法'], 'CCI121Y1031': ['哲學基本問題', '思維方法'], 'CCI122Y1031': ['哲學基本問題', '思維方法'], 'CCI211Y1031': ['藝術概論', '美學素養'], 'CCI212Y1031': ['藝術概論', '美學素養'], 'CCI213Y1031': ['藝術概論', '美學素養'], 'CCI251Y1031': ['台灣藝術史', '美學素養'], 'CCI261Y1031': ['西洋藝術史概論', '美學素養'], 'CCI262Y1031': ['西洋藝術史概論', '美學素養'], 'CCO311Y1031': ['法律與人生', '公民素養'], 'CCO312Y1031': ['法律與人生', '公民素養'], 'CCO351Y1031': ['服務學習', '公民素養'], 'CCO352Y1031': ['服務學習', '公民素養'], 'CCO354Y10

{'CCC511Y1031': ['科普名著導讀', '科學素養'], 'CCC512Y1031': ['科普讀物導讀', '科學素養'], 'CCC514Y1031': ['科普讀物導讀', '科學素養'], 'CCC541Y1031': ['科技與社會', '科學素養'], 'CCC542Y1031': ['科技與社會', '科學素養'], 'CCC543Y1031': ['科技與社會', '科學素養'], 'CCC544Y1031': ['科技與社會', '科學素養'], 'CCC611Y1031': ['環境倫理', '倫理素養'], 'CCC631Y1031': ['科技倫理', '倫理素養'], 'CCC632Y1031': ['科技倫理', '倫理素養'], 'CCC651Y1031': ['企業倫理', '倫理素養'], 'CCC652Y1031': ['企業倫理', '倫理素養'], 'CCC671Y1031': ['職場倫理', '倫理素養'], 'CCC672Y1031': ['職場倫理', '倫理素養'], 'CCI111Y1031': ['邏輯思維與論證', '思維方法'], 'CCI112Y1031': ['邏輯思維與論證', '思維方法'], 'CCI113Y1031': ['邏輯思維與論證', '思維方法'], 'CCI121Y1031': ['哲學基本問題', '思維方法'], 'CCI122Y1031': ['哲學基本問題', '思維方法'], 'CCI211Y1031': ['藝術概論', '美學素養'], 'CCI212Y1031': ['藝術概論', '美學素養'], 'CCI213Y1031': ['藝術概論', '美學素養'], 'CCI251Y1031': ['台灣藝術史', '美學素養'], 'CCI261Y1031': ['西洋藝術史概論', '美學素養'], 'CCI262Y1031': ['西洋藝術史概論', '美學素養'], 'CCO311Y1031': ['法律與人生', '公民素養'], 'CCO312Y1031': ['法律與人生', '公民素養'], 'CCO351Y1031': ['服務學習', '公民素養'], 'CCO352Y1031': ['服務學習', '公民素養'], 'CCO354Y10